In [1]:
import joblib
import pandas as pd

# Load the joblib file
data = joblib.load(r'C:\Users\Dwaipayan C\Downloads\Monthly_Income_Estimation_Target_Encoded_Artifacts_companytrgencode.joblib')

# Ensure that the loaded data is a DataFrame
if isinstance(data, pd.DataFrame):
    # Save the DataFrame to a CSV file
    data.to_csv('output_file_name.csv', index=False)
    print("Data has been successfully converted to CSV.")
else:
    print("Loaded data is not a DataFrame. Please ensure your joblib file contains a DataFrame.")


Data has been successfully converted to CSV.


In [4]:
import joblib
import pandas as pd
import zipfile

# Load the joblib file
data = joblib.load(r'C:\Users\Dwaipayan C\Downloads\Monthly_Income_Estimation_Target_Encoded_Artifacts_companytrgencode.joblib')

# Ensure that the loaded data is a DataFrame
if isinstance(data, pd.DataFrame):
    # Save the DataFrame to a CSV file
    csv_file = 'output_file_name.csv'
    data.to_csv(csv_file, index=False)
    
    # Compress the CSV file into a ZIP file
    zip_file = 'output_file_name.zip'
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as z:
        z.write(csv_file)
    
    print("Data has been successfully converted to CSV and compressed into a ZIP file.")
else:
    print("Loaded data is not a DataFrame. Please ensure your joblib file contains a DataFrame.")


Data has been successfully converted to CSV and compressed into a ZIP file.


# Step1

In [ ]:
import pandas as pd

# Load your data
df = pd.read_csv(r'C:\Users\Dwaipayan C\OneDrive\Learning\Projects\Income_Estimation\output_file_name.csv')

# Preview the data
print(df.head())
}

    global_mean  smoothing_factor       loan_company_name  \
0  30892.506245                15                     NaN   
1  30892.506245                15  Expert Security Agency   
2  30892.506245                15  leekie enterprises inc   
3  30892.506245                15          Mangiboa Store   
4  30892.506245                15           Fruits Vendor   

   freq_encodedcompanyName  category_mean_companyName  \
0                      NaN                        NaN   
1                      1.0               25000.000000   
2                      1.0               28000.000000   
3                      1.0               15000.000000   
4                      3.0               28333.333333   

   category_size_companyName  target_encodedcompanyName  \
0                        NaN               30892.506245   
1                        1.0               30524.224605   
2                        1.0               30711.724605   
3                        1.0               29899.22460

In [8]:
len(df['loan_company_name'].drop_duplicates().unique())

64771

# Step2

In [9]:
import re

# Function to clean company names
def preprocess_name(name):
    name = str(name).lower()  # Convert to lowercase
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    # Remove common suffixes
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
    pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
    name = re.sub(pattern, '', name)
    name = ' '.join(name.split())  # Remove extra spaces
    return name

# Apply preprocessing
df['clean_name'] = df['loan_company_name'].apply(preprocess_name)

# Preview cleaned names
print(df[['loan_company_name', 'clean_name']].head())


        loan_company_name              clean_name
0                     NaN                     nan
1  Expert Security Agency  expert security agency
2  leekie enterprises inc      leekie enterprises
3          Mangiboa Store          mangiboa store
4           Fruits Vendor           fruits vendor


# Step3

In [15]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Function to clean company names
def preprocess_name(name):
    name = str(name).lower()  # Convert to lowercase
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    # Remove common suffixes
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
    pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
    name = re.sub(pattern, '', name)
    name = ' '.join(name.split())  # Remove extra spaces
    return name

# Load your data

df = pd.read_csv(r'C:\Users\Dwaipayan C\OneDrive\Learning\Projects\Income_Estimation\output_file_name.csv')
# Apply preprocessing
df['clean_name'] = df['loan_company_name'].apply(preprocess_name)

# Get a list of unique cleaned names
unique_names = df['clean_name'].unique()

# Create a dictionary to map similar names
name_mapping = {}

for name in unique_names:
    if name in name_mapping.values():
        continue  # Skip if already mapped
    # Find matches with a similarity score above the threshold
    matches = process.extract(
        name,
        unique_names,
        scorer=fuzz.token_sort_ratio,  # Use fuzz.token_sort_ratio here
        score_cutoff=90  # Adjust this threshold as needed
    )
    # Map similar names to the representative name
    for match_name, score, _ in matches:
        name_mapping[match_name] = name

# Map the standardized names back to the DataFrame
df['standardized_name'] = df['clean_name'].map(name_mapping)

# Optionally, format the standardized names
df['standardized_name'] = df['standardized_name'].str.title()

# Save the cleaned data to a new CSV file
df.to_csv('standardized_company_names.csv', index=False)

print("Company names have been standardized and saved to 'standardized_company_names.csv'.")


Company names have been standardized and saved to 'standardized_company_names.csv'.


In [18]:
import pandas as pd
import jellyfish

# Function to get phonetic code
def get_phonetic_code(name):
    name = str(name)
    code = jellyfish.metaphone(name)
    return code

# Apply phonetic coding
df['phonetic_code'] = df['loan_company_name'].apply(get_phonetic_code)

# Map standardized names based on phonetic codes
df['standardized_name'] = df.groupby('phonetic_code')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name']].head())


        loan_company_name       standardized_name
0                     NaN                   Noona
1  Expert Security Agency  Expert Security Agency
2  leekie enterprises inc  leekie enterprises inc
3          Mangiboa Store          Mangiboa Store
4           Fruits Vendor           Fruits Vendor


In [19]:
# Save the cleaned data to a new csv file

df.to_csv("Phonetic_cleaned_company_names.csv", index=False)

# Untilizing Hashing of n-gram

In [20]:
import pandas as pd
import hashlib

# Function to generate n-gram hash
def ngram_hash(name, n=3):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    tokens = name.split()
    ngrams = [''.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    combined = ' '.join(ngrams)
    return hashlib.md5(combined.encode()).hexdigest()

# Apply n-gram hashing
df['ngram_hash'] = df['loan_company_name'].apply(ngram_hash)

# Map standardized names
df['standardized_name'] = df.groupby('ngram_hash')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name']].head())


        loan_company_name       standardized_name
0                     NaN          Mangiboa Store
1  Expert Security Agency  Expert Security Agency
2  leekie enterprises inc  leekie enterprises inc
3          Mangiboa Store          Mangiboa Store
4           Fruits Vendor          Mangiboa Store


In [21]:
df.to_csv("Ngram_cleaned_company_names.csv", index=False)

# Implementing Exact Matching with Extensive Normalization

In [22]:
import pandas as pd
import unicodedata

# Function to perform aggressive normalization
def aggressive_normalize(name):
    name = str(name).lower()
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\d+', '', name)  # Remove digits
    stop_words = ['the', 'and', 'of', 'in', 'a']
    tokens = [word for word in name.split() if word not in stop_words]
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'company', 'co', 'ltd', 'limited', 'plc']
    tokens = [word for word in tokens if word not in suffixes]
    name = ' '.join(sorted(tokens))
    return name.strip()

# Apply aggressive normalization
df['standardized_name'] = df['loan_company_name'].apply(aggressive_normalize)

# Preview the results
print(df[['loan_company_name', 'standardized_name']].head())


        loan_company_name       standardized_name
0                     NaN                     nan
1  Expert Security Agency  agency expert security
2  leekie enterprises inc      enterprises leekie
3          Mangiboa Store          mangiboa store
4           Fruits Vendor           fruits vendor


In [23]:
df.to_csv("Aggressive_cleaned_company_names.csv", index=False)

# 5. Clustering with String Similarity

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

# Preprocess names
def preprocess(name):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    return name

df['clean_name'] = df['loan_company_name'].apply(preprocess)

# Vectorize names
vectorizer = TfidfVectorizer(min_df=1, analyzer='char', ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(df['clean_name'])

# Cluster names using DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
dbscan.fit(tfidf_matrix)

# Assign cluster labels
df['cluster'] = dbscan.labels_

# Map standardized names
df['standardized_name'] = df.groupby('cluster')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name']].head())


        loan_company_name       standardized_name
0                     NaN                    None
1  Expert Security Agency  Expert Security Agency
2  leekie enterprises inc  Expert Security Agency
3          Mangiboa Store  Expert Security Agency
4           Fruits Vendor  Expert Security Agency
